<a href="https://colab.research.google.com/github/sanyamChaudhary27/ML_models_from_scratch/blob/main/Lasso_ElasticNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [30]:
class LASSO:
    def __init__(self, n_iterations = 100, learning_rate = 0.1, lam = 0.1):
        self.n_iterations = n_iterations
        self.learning_rate = learning_rate
        self.lam = lam
        self.intercept = 0
        self.weights = None

    def fit(self, X, y):
        self.X = X
        self.y = y.to_numpy().flatten()
        n_samples, n_features = self.X.shape
        self.weights = np.zeros(n_features)
        self.intercept = 0

        # Batch gradient Descent with L1 (Lasso) regularization
        for _ in range(self.n_iterations):
            y_pred = self.predict(self.X)
            error = y_pred - self.y
            dw_data = (1/n_samples) * np.dot(self.X.T, error)
            dw_l1 = self.lam * np.sign(self.weights)
            self.weights -= self.learning_rate * (dw_data + dw_l1)
            db = (1/n_samples) * np.sum(error)
            self.intercept -= self.learning_rate * db

    def predict(self, X):
        if self.weights is None or self.intercept is None:
            raise RuntimeError("Model has not been fitted yet. Call .fit(X, y) first.")
        return np.dot(X, self.weights) + self.intercept

In [31]:
class ElasticNet:
    def __init__(self, n_iterations=100, learning_rate=0.1, lam=0.1, alpha=0.5):
        # lam is the overall regularization strength
        # alpha is the mixing parameter between L1 and L2 (0 for Ridge, 1 for Lasso)
        self.n_iterations = n_iterations
        self.learning_rate = learning_rate
        self.lam = lam
        self.alpha = alpha
        self.intercept = 0
        self.weights = None

    def fit(self, X, y):
        self.X = X
        self.y = y.to_numpy().flatten()
        n_samples, n_features = self.X.shape

        self.weights = np.zeros(n_features)
        self.intercept = 0

        for _ in range(self.n_iterations):
            y_pred = np.dot(self.X, self.weights) + self.intercept
            error = y_pred - self.y
            dw_data = (1/n_samples) * np.dot(self.X.T, error)
            dw_l1 = self.lam * self.alpha * np.sign(self.weights)
            dw_l2 = self.lam * (1 - self.alpha) * 2 * self.weights
            self.weights -= self.learning_rate * (dw_data + dw_l1 + dw_l2)
            db = (1/n_samples) * np.sum(error)
            self.intercept -= self.learning_rate * db

    def predict(self, X):
        if self.weights is None or self.intercept is None:
            raise RuntimeError("Model has not been fitted yet. Call .fit(X, y) first.")
        return np.dot(X, self.weights) + self.intercept

In [8]:
from sklearn.datasets import fetch_california_housing

In [11]:
data = fetch_california_housing()

In [14]:
X = pd.DataFrame(data.data)
X.columns = data.feature_names
y = pd.DataFrame(data.target)
y.columns = data.target_names

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state =42)

In [18]:
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()

In [27]:
X_train = Scaler.fit_transform(X_train)
X_val = Scaler.transform(X_val)

In [33]:
lasso_model = LASSO(n_iterations=1000, learning_rate=0.01)
elastic_model = ElasticNet(n_iterations=1000, learning_rate=0.01)

In [34]:
lasso_model.fit(X_train, y_train)
elastic_model.fit(X_train, y_train)

In [35]:
lasso_preds = lasso_model.predict(X_val)
elastic_preds = elastic_model.predict(X_val)

In [36]:
from sklearn.metrics import mean_squared_error

In [37]:
def rmse(y, y_preds):
    return mean_squared_error(y, y_preds)

In [39]:
print(rmse(y_val, lasso_preds))

0.6794924441633325


In [40]:
print(rmse(y_val, elastic_preds))

0.6524712990482402


### Conclusion

This notebook implemented custom LASSO and ElasticNet regression models from scratch and evaluated their performance on the California Housing dataset. Both models were trained on the preprocessed training data and evaluated on the validation set using Root Mean Squared Error (RMSE).

**Performance Summary:**
*   **LASSO Model RMSE:** 0.679
*   **ElasticNet Model RMSE:** 0.652

ElasticNet achieved a slightly lower RMSE (0.652) compared to LASSO (0.679) on the validation set, indicating slightly better predictive accuracy for this dataset. This suggests that the combination of L1 and L2 regularization in ElasticNet was marginally more effective than pure L1 regularization (LASSO) for this specific problem.

## Summary:

### Data Analysis Key Findings
*   Custom LASSO and ElasticNet regression models were implemented and evaluated on the California Housing dataset.
*   The Root Mean Squared Error (RMSE) for the LASSO model was 0.679.
*   The Root Mean Squared Error (RMSE) for the ElasticNet model was 0.652.
*   ElasticNet achieved a slightly lower RMSE (0.652) compared to LASSO (0.679), indicating marginally better predictive accuracy for this dataset.

### Insights or Next Steps
*   The results suggest that combining L1 and L2 regularization (ElasticNet) was slightly more effective than pure L1 regularization (LASSO) for this specific regression problem.
*   Further hyperparameter tuning for both models could potentially lead to even better performance, or exploring other regularization techniques might offer additional improvements.
